<a href="https://colab.research.google.com/github/EilieYoun/Narnia-Edu/blob/main/Lecture/240822_kaist/02_DeepSDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2024 KAIST 생성AI 실습 :  DeepSDF

* 날짜:
* 이름:



## **(0) Environment Setup**
---

### **| Utils**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LinearSegmentedColormap
import random

import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

In [ ]:
def set_seed(seed=42):
    # Python의 내장 난수 생성기 시드 설정
    random.seed(seed)

    # NumPy 난수 생성기 시드 설정
    np.random.seed(seed)

    # PyTorch 난수 생성기 시드 설정 (CPU)
    torch.manual_seed(seed)

    # PyTorch 난수 생성기 시드 설정 (GPU)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # 모든 GPU에 대해 시드 설정

    # CuDNN 설정
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def draw_3d_scatter(data, n=50000):
    # 샘플링된 데이터
    samples = data[np.random.choice(data.shape[0], n, replace=False)]

    x, y, z, sdf = samples[:, 0], samples[:, 1], samples[:, 2], samples[:, 3]

    # SDF 값이 0 이하인 데이터만 필터링 (첫 번째 플롯용)
    mask_sdf_negative = sdf <= 0
    x_neg = x[mask_sdf_negative]
    y_neg = y[mask_sdf_negative]
    z_neg = z[mask_sdf_negative]
    sdf_neg = sdf[mask_sdf_negative]

    # 3D 시각화
    fig = plt.figure(figsize=(20, 6))

    # 1st subplot: 3D scatter plot for SDF <= 0
    ax1 = fig.add_subplot(131, projection='3d')
    custom_blue = LinearSegmentedColormap.from_list("custom_blue", ["#0000ff", "white"])

    scatter = ax1.scatter(x_neg, y_neg, z_neg, c=sdf_neg, cmap=custom_blue, marker='.')
    fig.colorbar(scatter, ax=ax1, label='SDF Value')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title('3D Scatter Plot (SDF <= 0)')

    # 2nd subplot: 2D Visualization at z ≈ 0 (모든 데이터, SDF 값 클램핑)
    z_threshold = 0.05  # z=0에서의 허용 오차
    mask_z = np.abs(z) < z_threshold  # z 값이 0에 가까운 것만 선택
    x_filtered_z = x[mask_z]
    y_filtered_z = y[mask_z]
    sdf_filtered_z = np.clip(sdf[mask_z], -0.2, 0.2)  # SDF 값을 -0.2에서 0.2로 제한

    ax2 = fig.add_subplot(132)
    sc1 = ax2.scatter(x_filtered_z, y_filtered_z, c=sdf_filtered_z, cmap='seismic', s=50, edgecolor=None)
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_title('2D Visualization at z ≈ 0 (clamped SDF)')
    fig.colorbar(sc1, ax=ax2, label='SDF Value')

    # 3rd subplot: 2D Visualization at x ≈ 0 (모든 데이터, SDF 값 클램핑)
    x_threshold = 0.05  # x=0에서의 허용 오차
    mask_x = np.abs(x) < x_threshold  # x 값이 0에 가까운 것만 선택
    y_filtered_x = y[mask_x]
    z_filtered_x = z[mask_x]
    sdf_filtered_x = np.clip(sdf[mask_x], -0.2, 0.2)  # SDF 값을 -0.2에서 0.2로 제한

    ax3 = fig.add_subplot(133)
    sc2 = ax3.scatter(y_filtered_x, z_filtered_x, c=sdf_filtered_x, cmap='seismic', s=50, edgecolor=None)
    ax3.set_xlabel('Y')
    ax3.set_ylabel('Z')
    ax3.set_title('2D Visualization at x ≈ 0 (clamped SDF)')
    fig.colorbar(sc2, ax=ax3, label='SDF Value')

    plt.show()


def draw_2dgrid(grid, mn=-4, mx=4):
    plt.figure(figsize=(8,8))
    plt.imshow(grid, cmap='RdBu', vmin=mn, vmax=mx, extent=[mn, mx, mn, mx])
    plt.colorbar( shrink=0.8)
    plt.show()

def compare_2dplot(points, real_sdf, pred_sdf, x_threshold=0.2):
    """
    x ≈ 0 근처에서의 2D 시각화를 수행합니다. 각 샘플에 대해 실제 값과 예측 값을 시각화합니다.

    Parameters:
    points (torch.Tensor): (2, n, 3) 크기의 3D 좌표 배열, 각 행이 [x, y, z] 형태.
    real_sdf (torch.Tensor): (2, n) 크기의 실제 SDF 값 배열.
    pred_sdf (torch.Tensor): (2, n) 크기의 예측된 SDF 값 배열.
    x_threshold (float): x=0에서의 허용 오차.
    """
    plt.figure(figsize=(18, 4))

    for i in range(2):
        # 현재 샘플의 데이터를 추출
        current_points = points[i]
        current_real_sdf = real_sdf[i]
        current_pred_sdf = pred_sdf[i]

        # x ≈ 0에서의 데이터 필터링
        x = current_points[:, 0]  # x 좌표
        y = current_points[:, 1]  # y 좌표
        z = current_points[:, 2]  # z 좌표

        mask_x = np.abs(x) < x_threshold  # x 값이 0에 가까운 것만 선택
        y_filtered_x = y[mask_x]
        z_filtered_x = z[mask_x]
        real_sdf_filtered_x = np.clip(current_real_sdf[mask_x], -0.2, 0.2)  # 실제 SDF 값을 -0.2에서 0.2로 제한
        pred_sdf_filtered_x = np.clip(current_pred_sdf[mask_x], -0.2, 0.2)  # 예측된 SDF 값을 -0.2에서 0.2로 제한

        # subplot(141) 또는 subplot(143): Real SDF for sample i
        ax_real = plt.subplot(1, 4, i * 2 + 1)
        sc_real = ax_real.scatter(y_filtered_x, z_filtered_x, c=real_sdf_filtered_x, cmap='seismic', s=50, edgecolor=None)
        ax_real.set_xlabel('Y')
        ax_real.set_ylabel('Z')
        ax_real.set_title(f'Sample {i+1} - Real SDF')
        plt.colorbar(sc_real, ax=ax_real, label='SDF Value')

        # subplot(142) 또는 subplot(144): Predicted SDF for sample i
        ax_pred = plt.subplot(1, 4, i * 2 + 2)
        sc_pred = ax_pred.scatter(y_filtered_x, z_filtered_x, c=pred_sdf_filtered_x, cmap='seismic', s=50, edgecolor=None)
        ax_pred.set_xlabel('Y')
        ax_pred.set_ylabel('Z')
        ax_pred.set_title(f'Sample {i+1} - Predicted SDF')
        plt.colorbar(sc_pred, ax=ax_pred, label='SDF Value')

    plt.tight_layout()
    plt.show()

In [ ]:
set_seed(0)

### **| Data download**

In [ ]:
!gdown --folder https://drive.google.com/drive/u/0/folders/130c6Iq47wCbYS6jAQwsdOh_9cNY4TbZ1


## **(1) What is SDF?**

**Signed Distance Function (SDF)**는 3D 그래픽스 및 컴퓨터 비전에서 자주 사용되는 개념으로, 주어진 공간의 각 점에서 가장 가까운 표면까지의 거리와 그 방향을 나타냅니다. SDF는 다음과 같은 특성을 가지고 있습니다:

- 거리 계산: SDF는 공간 내의 임의의 점에서 표면까지의 최단 거리를 계산합니다.
- 부호: SDF의 값은 부호를 가집니다. 표면 내부의 점은 음의 값을 가지며, 표면 외부의 점은 양의 값을 가집니다. 표면 자체의 점은 0의 값을 가집니다.
- 표면 표현: SDF는 임의의 3D 형상을 수학적으로 간결하게 표현할 수 있으며, 복잡한 형상도 간단하게 다룰 수 있습니다.

이러한 특성 때문에 SDF는 충돌 감지, 형태 최적화, 3D 모델링 및 렌더링 등 다양한 응용 분야에서 활용됩니다. 예를 들어, 3D 모델의 표면을 정의하고 그 표면과의 거리를 계산하여 물리적 충돌을 감지할 수 있습니다.

이번 시간에는 SDF의 개념을 이해하기 위해 원을 예제로 사용해 보겠습니다. 원은 2D 공간에서 간단한 형상으로, 각 점에서 원의 경계까지의 거리를 계산하여 SDF를 쉽게 설명할 수 있습니다. 다음 섹션에서는 원의 SDF를 정의하고, 다양한 점들에 대해 SDF 값을 계산하여 시각화하는 과정을 다루겠습니다. 이를 통해 SDF가 어떻게 작동하는지에 대한 직관적인 이해를 돕고자 합니다.









### **| Define 2D Circle**

여기서는 `Circle` 클래스를 정의합니다. 이 클래스는 중심 좌표 `(cx, cy)`와 반지름 `(r)`을 가지며, `sdf` 메서드는 주어진 점 `(px, py)`에 대한 **Signed Distance Function** 값을 계산합니다. 이 함수는 점과 원의 경계 사이의 거리를 나타냅니다.



In [ ]:
class Circle():
    def __init__(self, cx, cy, r):
        self.cx = cx
        self.cy = cy
        self.r = r

    def sdf(self, px, py):
        point = np.float_([px, py])
        center = np.float_([self.cx, self.cy])
        sdf = np.linalg.norm(point - center) - self.r
        return sdf

중심 좌표가 `(0, 0)` 이고 반지름이 2 인 원을 정의합니다.

In [ ]:
circle = Circle(0, 0, 2)

### **| Get points**

이 함수는 지정된 범위 `(mn~mx)` 와 해상도 `(resolution)` 에 따라 2D 평면의 격자점들을 생성합니다. 각 점의 좌표는 리스트로 저장되고, 이 리스트는 나중에 Numpy 배열로 변환됩니다. 이 배열은 후속 SDF 계산에 사용됩니다.

In [ ]:
def get_points(mn=-5, mx=5, resolution=200):
    points = []
    for px in np.linspace(mn, mx, resolution):
        for py in np.linspace(mn, mx, resolution):
            points.append([px, py])
    points = np.array(points, dtype=np.float32)
    return points

범위 (-5~5) 이며 해상도 200 인 격자점들을 생성합니다.

In [ ]:
points = get_points(-5, 5, 200)
print(points.shape)

### **| Get SDF Values**

앞에서 얻은 격자점에 대하여 `Circle` 클래스의 `sdf` 메서드를 호출하여 SDF 값을 계산합니다.

In [ ]:
sdf_values = np.array([circle.sdf(px, py) for px, py in zip(points[:,0], points[:,1])] )
print(sdf_values.shape)

### **| Visualize**

계산된 SDF 값들을 2D 그리드 형태로 재배열하여 시각화합니다. `draw_grid` 함수를 사용하여 그리드를 시각화할 수 있습니다.

In [ ]:
sdf_grid = np.reshape(sdf_values, (200, 200))
print(sdf_grid.shape)
draw_2dgrid(sdf_grid)

SDF 값이 양수인 영역을 1로, 음수인 영역을 0으로 설정하여 `voxel grid`를 생성할 수 있습니다. SDF와 voxel을 차이를 시각적으로 비교해 봅니다.

In [ ]:
voxel_grid = np.where(sdf_grid>0, 1., 0.)
print(voxel_grid.shape)
draw_2dgrid(voxel_grid)

## **(2) Dataset**

이 섹션에서는 3D 공간에서의 SDF(Signed Distance Function) 개념을 더욱 구체적으로 이해하기 위해, 이미 생성된 sphere(구체)와 cylinder(원통) 두 개의 3D 객체를 예제로 사용합니다. 우리는 Numpy 형식의 예제 데이터셋을 불러와, 3D 공간에서의 SDF를 시각화하고 분석할 것입니다


### **| 데이터 확인**

In [ ]:
data_paths=['dataset/sphere.npy', 'dataset/cylinder.npy']

x1 = np.load(data_paths[0])
print(x1.shape, x1[0])

x2 = np.load(data_paths[1])
print(x2.shape, x2[0])

데이터를 시각화 합니다.

In [ ]:
draw_3d_scatter(x1)
draw_3d_scatter(x2)

### **| Prepair dataloader**


데이터 경로로부터 넘파이 배열을 불러오고, 좌표값, SDF값 데이터 인덱스를 추출하고 준비하는 과정을 설명합니다.
$$ TRAIN := \{((x, y, z), s) : SDF(x, y, z) = s\} $$

In [ ]:
class dataset(Dataset):
    def __init__(self, data_paths, npoints=2048):
        self.data_paths = data_paths #[item for item in os.listdir(path) if item[-4:] == '.npy']
        self.npoints = npoints

    def __len__(self):
        return len(self.data_paths)

    # 45% inside, 45% outside, 10% sphere
    def __getitem__(self, idx):
        path = self.data_paths[idx]
        data = np.load(path)
        samples = data[np.random.choice(data.shape[0], self.npoints, replace=False)]
        point = samples[...,:3]
        sdf = samples[...,-1]
        return point, sdf, idx

In [ ]:
num_points = 1024

train_dataset = dataset(data_paths, num_points)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 2, shuffle=True)

In [ ]:
for batch_points, batch_sdf, batch_idx in train_loader:
    print(batch_points.shape, batch_sdf.shape, batch_idx.shape)

## **(3) Model**

3D 객체의 SDF(Signed Distance Function)를 학습하고 예측하기 위한 모델 구조를 정의합니다. 이 모델은 두 가지 주요 구성 요소로 이루어져 있습니다: `ShapeCodeEmbedder`와 `Net`입니다.

`ShapeCodeEmbedder`는 각 3D 객체에 고유한 임베딩(embedding)을 제공하며, 이는 객체의 모양을 잠재 공간(latent space)에서 나타내는 벡터입니다. `Net`은 이 임베딩 벡터와 3D 좌표를 입력으로 받아, 해당 좌표에서의 SDF 값을 예측하는 역할을 합니다. 이 두 구성 요소는 함께 작동하여 다양한 3D 객체에 대해 SDF를 효과적으로 모델링할 수 있습니다.



### **| Define Embedder**

`ShapeCodeEmbedder`는 모델이 학습하는 각 3D 객체에 대해 고유한 임베딩 벡터를 생성하는 역할을 합니다. 임베딩 벡터는 잠재 공간(latent space)에서 객체의 모양을 표현하는데 사용됩니다. 이 클래스는 다음과 같은 주요 기능을 가지고 있습니다:

- 임베딩 초기화: 임베딩 벡터는 가우시안 분포를 사용해 무작위로 초기화됩니다. 초기화는 `reset_parameters` 메서드를 통해 수행되며, 임베딩 벡터가 적절한 범위 내에서 시작되도록 합니다.
- 임베딩 인덱싱: `forward` 메서드는 주어진 객체 인덱스에 해당하는 임베딩 벡터를 반환합니다. 이 벡터는 이후 모델의 입력으로 사용됩니다.

In [ ]:
class ShapeCodeEmbedder(nn.Module):
    def __init__(self, N, dim):
        super(ShapeCodeEmbedder, self).__init__()
        self.N = N
        self.dim = dim
        self.embed_params = nn.Parameter(torch.Tensor(N, dim))
        self.reset_parameters()

    def reset_parameters(self):
        init_std = 1.0 / np.sqrt(self.dim)

        torch.nn.init.normal_(
            self.embed_params.data,
            0.0,
            init_std,
        )

    def forward(self, idx):
        batch_embed = self.embed_params[idx]
        batch_embed = batch_embed.unsqueeze(1)

        return batch_embed

In [ ]:
N = 2
emb_dims = 16

embedder = ShapeCodeEmbedder(N, emb_dims)
batch_latent = embedder(batch_idx)
print(batch_latent.shape)

In [ ]:
batch_latent_codes = batch_latent.repeat(1,num_points,1)
batch_latent_codes.shape, batch_points.shape

### **| Network Architecture**

![](https://github.com/EilieYoun/Narnia-Edu/blob/main/Lecture/imgs/2408_KAIST_02_02.png?raw=true)

`Net`은 주어진 임베딩 벡터와 3D 좌표를 결합하여, 해당 좌표에서의 SDF 값을 예측하는 역할을 합니다. 이 신경망은 다음과 같은 구조를 가지고 있습니다:

- 입력 계층: Net은 임베딩 벡터와 3D 좌표를 입력으로 받아, 이를 첫 번째 완전 연결층(fc1)에 전달합니다. 이 계층은 임베딩과 좌표를 결합하여 잠재 공간에서 더 복잡한 특징을 학습합니다.
- 은닉 계층: fc1 계층의 출력은 두 번째 완전 연결층(fc2)으로 전달되어, 더 깊은 특징을 학습하게 됩니다.
- 출력 계층: 마지막으로, fc2 계층의 출력은 fc3로 전달되며, 이 계층에서 최종적으로 SDF 값이 예측됩니다. 예측된 SDF 값은 tanh 활성화 함수를 통해 제한된 범위로 정규화됩니다.


In [ ]:
class Net(nn.Module):

    def __init__(self, lc_dim = 16, h_dim = 50):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(lc_dim + 3, h_dim)
        self.fc2 = nn.Linear(h_dim, h_dim)
        self.fc3 = nn.Linear(h_dim, 1)

    def forward(self, x):
        x = torch.Tensor(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        return x


net = Net()
print(net)

네트워크의 인풋과 아웃풋을 확인합니다.

In [ ]:
model_inputs = torch.cat([batch_latent_codes, batch_points], axis = 2)
with torch.no_grad():
    model_outputs = net(model_inputs)
model_inputs.shape, model_outputs.shape

현재 네트워크의 성능을 시각적으로 평가합니다

In [ ]:
preds_sdf = model_outputs.numpy()[:,:,0]
compare_2dplot( batch_points, batch_sdf, preds_sdf)

### **| Model Training**

- `optim.Adam`: 확률적 경사 하강법(Stochastic Gradient Descent, SGD)을 기반으로 하며, 학습 과정에서 학습률을 적응적으로 조정하는 기능을 포함합니다.
- `StepLR`: 학습률 스케줄러를 설정하여 학습률을 점진적으로 감소시킵니다.
- `compare_2dplot`: 일정 에폭마다 학습된 모델의 성능을 고정된 샘플 데이터로 평가합니다.

In [ ]:
def train_model(net, embedder, train_loader,epochs=2000, device='cuda'):
    embedder.to(device)
    net.to(device)

    optim = torch.optim.Adam(net.parameters(), lr=0.001, betas=[0.9,0.999], eps=1.0e-12, weight_decay=0.1, amsgrad=False)
    scheduler = StepLR(optim, step_size=500, gamma=0.5)

    optim_emb = torch.optim.Adam(embedder.parameters(), lr=0.001, betas=[0.9,0.999], eps=1.0e-12, weight_decay=0.1, amsgrad=False)
    scheduler_emb = StepLR(optim_emb, step_size=500, gamma=0.5)

    # prepair inferece sample
    for bidx, (sample_points, sample_sdf, sample_idx) in enumerate(train_loader):
        sample_points = sample_points.to(device)
        sample_sdf = sample_sdf.to(device)
        sample_codes = embedder(sample_idx).repeat(1,num_points,1).to(device)
        sample_inputs = torch.cat([sample_codes, sample_points], axis = 2).to(device)
        break

    # train
    with tqdm(range(epochs)) as pbar:
        losses = []
        net.train()
        embedder.train()
        loss = nn.L1Loss()

        for epoch in pbar:
            for bidx, (points, sdf_gt, idx) in enumerate(train_loader):
                optim.zero_grad()
                optim_emb.zero_grad()

                points = points.to(device)
                sdf_gt = sdf_gt.to(device)
                latent_emb = embedder(idx).repeat(1,num_points,1).to(device)
                latent_codes = torch.cat([latent_emb, points], axis = 2).to(device)

                model_output = net(latent_codes).squeeze()

                train_loss = torch.sqrt(torch.sum((sdf_gt - model_output)**2))
                pbar.set_description("loss: {:.6f}".format(train_loss.item()))

                # backward
                train_loss.backward()
                optim.step()
                scheduler.step()

                optim_emb.step()
                scheduler_emb.step()
                losses.append(train_loss.item())

            if epoch %(epochs//10)==0:
                with torch.no_grad():
                    sample_outputs = net(sample_inputs)
                compare_2dplot( sample_points.cpu().numpy(), sample_sdf.cpu().numpy(), sample_outputs.cpu().numpy())

    compare_2dplot( sample_points.cpu().numpy(), sample_sdf.cpu().numpy(), sample_outputs.cpu().numpy())


    return net, embedder, losses

모델 학습

In [ ]:
net, embedder, losses = train_model(net, embedder, train_loader)

학습 과정을 시각화합니다.

In [ ]:
plt.figure(figsize=(12,3))
plt.title('Train Losses')
_=plt.plot(losses)